___
**ETL**

IvanMF888
___

In [7]:
# Librerias 
import pandas as pd
import numpy as np 
import string
import hashlib
import os

from faker import Faker
# Inicializa el generador de datos sintéticos para crear perfiles de usuario o registros de prueba
fake = Faker()
# Establece una semilla fija para garantizar que los datos generados sean deterministas y reproducibles
Faker.seed(420)

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display # Necesario para ver tablas

In [8]:
# Define la ruta hacia la carpeta de datos crudos utilizando rutas relativas para asegurar portabilidad entre sistemas
ruta_raw = os.path.join("..", "data", "raw")

# Estructura temporal para almacenar los DataFrames individuales antes de la consolidación
df_procesado = {}

# Itera sobre el contenido del directorio para identificar archivos de datos
for archivo in os.listdir(ruta_raw):

    # Filtra exclusivamente archivos con extensión CSV para evitar errores con archivos ocultos o de sistema
    if archivo.endswith(".csv"):
        ruta_completa = os.path.join(ruta_raw, archivo)
        
        try:
            # Intento de carga inicial bajo el estándar universal UTF-8
            df_temp = pd.read_csv(ruta_completa)
        except UnicodeDecodeError:
            # Fallback para archivos generados en entornos Windows o con codificación regional (soporte para ñ y tildes)
            df_temp = pd.read_csv(ruta_completa, encoding='latin-1')
            
        # Almacena el DataFrame en el diccionario usando el nombre del archivo como clave para trazabilidad
        df_procesado[archivo] = df_temp
        
        # Reporte de estructura para validar la integridad de la carga individual
        print(f"\nDimensiones: {archivo}\n{df_temp.shape}")
    
# Integra todos los DataFrames recolectados en una única estructura tabular
# ignore_index=True evita que se repitan índices de los archivos originales en el resultado final
df_procesado = pd.concat(df_procesado.values(), ignore_index=True)

# Resumen técnico del dataset final para verificar tipos de datos y valores nulos
print(df_procesado.info())

# ||------------------------------------------------------------||
# 1. ANONIMIZACIÓN (AGENTES Y SKILLS)
print("[+] Anonimizando datos sensibles...")

# Se verifica la existencia de la columna para evitar errores en ejecuciones parciales
if 'Nombre del RAC' in df_procesado.columns:
    nombres_unicos = df_procesado['Nombre del RAC'].unique()
    # Se crea un diccionario de mapeo para sustituir nombres reales por identificadores genéricos (Agente_0001)
    # Esto protege la identidad del personal manteniendo la integridad referencial para el análisis
    mapa_agentes = {nombre: f"Agente_{i+1:04d}" for i, nombre in enumerate(nombres_unicos)}
    df_procesado['Agente_ID'] = df_procesado['Nombre del RAC'].map(mapa_agentes)

# --- B) SKILLS ---
if 'Split/Skill' in df_procesado.columns:
    skills_unicos = df_procesado['Split/Skill'].unique()
#    skills_unicos.sort()

    letras = list(string.ascii_uppercase)
    # Función para generar etiquetas alfabéticas (A, B... AA, AB) similar a la nomenclatura de columnas en hojas de cálculo
    def get_skill_label(i):
        if i < 26: return f"Skill_{letras[i]}"
        return f"Skill_{letras[i//26 - 1]}{letras[i%26]}"

    mapa_skills = {skill: get_skill_label(i) for i, skill in enumerate(skills_unicos)}
    df_procesado['Skill_ID'] = df_procesado['Split/Skill'].map(mapa_skills)
    
    # print("Matriz de Skills:", list(mapa_skills.items())[:3])

# --- C) BORRADO DE ORIGINALES ---
# Se eliminan las columnas de identificación personal y datos crudos para asegurar que el dataset resultante sea anónimo
df_procesado.drop(columns=['Nombre del RAC', 'Login ID', 'Split/Skill'], inplace=True, errors='ignore')

print("[+] Anonimización completa.")

# ||------------------------------------------------------------||
# 2. LIMPIEZA DE FECHAS
print("[+] Ajustando Fechas y KPIs...")

# Estandarización del formato de fecha para permitir operaciones de series temporales
# df_procesado['Fecha'] = pd.to_datetime(df_procesado['Fecha'], errors='coerce')

# KPIS
# Cálculo del tiempo total de gestión (Total Handle Time) sumando conversación, espera y trabajo administrativo posterior
df_procesado['Total_Handle_Time'] = (
    df_procesado['Tiempo Total de Conversación'] + 
    df_procesado['Tiempo Total Hold'] + 
    df_procesado['Tiempo Total ACW']
)

# Cálculo del AHT (Average Handle Time). Se utiliza np.where para prevenir errores de división por cero
# en registros donde no se atendieron llamadas, asignando un valor neutro (0)
df_procesado['AHT'] = np.where(
    df_procesado['Llamadas Atendidas'] > 0,
    df_procesado['Total_Handle_Time'] / df_procesado['Llamadas Atendidas'],
    0
)

# ||------------------------------------------------------------||
# 3. TRANSFORMACIÓN DE HORARIOS (Feature Engineering)
print("[+] Extrayendo Franjas Horarias (Método Blindado)...")

# Paso A: Convertir la columna a Datetime real (Pandas es inteligente)
# 'coerce' transforma errores (textos raros o vacíos) en NaT (Not a Time)
fechas_convertidas = pd.to_datetime(df_procesado['Tiempo Inicio'], errors='coerce')

# Paso B: Extraer la hora
# Si la conversión funcionó, sacamos la hora (.dt.hour).
# Si falló (NaT), llenamos con -1 temporalmente.
df_procesado['Hora'] = fechas_convertidas.dt.hour.fillna(-1).astype(int)

# Paso C: Intento de rescate (Solo si el paso A falló mucho)
# A veces 'Tiempo Inicio' es solo texto "08:00:00" y to_datetime se confunde sin fecha.
if (df_procesado['Hora'] == -1).sum() > len(df_procesado) * 0.5:
    print("   [!] Aviso: to_datetime detectó muchos fallos. Intentando método de texto...")
    # Forzamos conversión a string, tomamos lo que está antes de los dos puntos
    # str.split(':').str[0] -> De "08:30:00" saca "08"
    # pd.to_numeric(..., errors='coerce') -> Convierte "08" en 8, y "nan" en NaN
    df_procesado['Hora'] = pd.to_numeric(
        df_procesado['Tiempo Inicio'].astype(str).str.split(':').str[0], 
        errors='coerce'
    ).fillna(-1).astype(int)

# Validación final en pantalla
horas_unicas = sorted(df_procesado['Hora'].unique())
print(f"   [+] Horas detectadas finalmente: {horas_unicas}")

if -1 in horas_unicas:
    print("   ⚠️ ADVERTENCIA: Todavía hay registros con hora -1 (Desconocida). Revisa si hay nulos en 'Tiempo Inicio'.")

# Crear etiquetas de Turno (Ahora sí funcionará)
def etiquetar_turno(h):
    if h == -1: return 'Desconocido'
    if 5 <= h < 15: return 'Matutino'     
    if 15 <= h < 23: return 'Vespertino'  
    return 'Nocturno/Mixto'

df_procesado['Turno'] = df_procesado['Hora'].apply(etiquetar_turno)
print("[+] Columnas 'Hora' y 'Turno' creadas con éxito.")

# ||------------------------------------------------------------||
# 4. LIMPIEZA DE COLUMNAS
# Definición de columnas que no aportan valor al análisis final o que ya han sido procesadas en nuevas métricas
# 4.1 LIMPIEZA EXTRA (Eliminar columnas vacías/basura)
print("[+] Eliminando columnas basura (Unnamed)...")
cols_basura = [c for c in df_procesado.columns if "Unnamed" in c]
df_procesado = df_procesado.drop(columns=cols_basura, errors='ignore')

columnas_innecesarias = [
    # 'Tiempo Inicio',          <-- Se preserva para análisis de granularidad temporal en herramientas de BI
    'Tiempo Fin',             
    'Llamadas Transferidas',
    'Otra hora', 
    'Tiempo con personal', 
    'Tiempo Total Abandono', 
    'Tiempo Total Transferencias', 
    'Llamadas en Hold', 
    'Llamadas en ring', 
    'Tiempo Total AUX'
]

# Reducción de la dimensionalidad del DataFrame para optimizar el uso de memoria y la velocidad de carga
df_procesado = df_procesado.drop(columns=columnas_innecesarias, errors='ignore')

# ||------------------------------------------------------------||
# 5. EXPORTACIÓN FINAL
# Definición de la ruta de salida y creación automática de directorios si no existen
ruta_salida = os.path.join("..", "data", "processed", "Base_ContactCenter_Master.csv")
os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)

# Persistencia de los datos limpios y transformados en formato CSV para su consumo posterior
df_procesado.to_csv(ruta_salida, index=False)

print(f"\n[+] Done!")
print(f"[+] Archivo guardado en: {ruta_salida}")
print(f"[+] Dimensiones finales: {df_procesado.shape}")
display(df_procesado.head())


Dimensiones: 3.-Acumulado tiempos-1.csv
(1044329, 36)

Dimensiones: 3.-Acumulado tiempos-2.csv
(607698, 32)

Dimensiones: T-252.csv
(994389, 32)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2646416 entries, 0 to 2646415
Data columns (total 36 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   Login ID                      int64  
 1   Nombre del RAC                object 
 2   Split/Skill                   object 
 3   Fecha                         object 
 4   Tiempo Inicio                 object 
 5   Tiempo Fin                    object 
 6   Llamadas Recibidas            int64  
 7   Llamadas abandonas            int64  
 8   Llamadas Atendidas            int64  
 9   Llamadas Transferidas         int64  
 10  Tiempo Total Ring             int64  
 11  Tiempo Total de Conversación  int64  
 12  Tiempo Total Hold             int64  
 13  Tiempo Total ACW              int64  
 14  Tiempo Total Avail            int64  
 15  Otra 

C:\Users\Ivan Die\AppData\Local\Temp\ipykernel_15436\2976726782.py:97: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  fechas_convertidas = pd.to_datetime(df_procesado['Tiempo Inicio'], errors='coerce')


   [+] Horas detectadas finalmente: [np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22)]
[+] Columnas 'Hora' y 'Turno' creadas con éxito.
[+] Eliminando columnas basura (Unnamed)...

[+] Done!
[+] Archivo guardado en: ..\data\processed\Base_ContactCenter_Master.csv
[+] Dimensiones finales: (2646416, 26)


,Fecha,Tiempo Inicio,Llamadas Recibidas,Llamadas abandonas,Llamadas Atendidas,Tiempo Total Ring,Tiempo Total de Conversación,Tiempo Total Hold,Tiempo Total ACW,Tiempo Total Avail,...,AUX 6 RETRO_CALIDAD,AUX 7 MEETING,AUX 8 PERSONALES,AUX 9 FALLA APLICATIVO,Agente_ID,Skill_ID,Total_Handle_Time,AHT,Hora,Turno
0,01/01/2024,09:00,0,0,0,0,0,0,0,1661,...,0,0,0,0,Agente_0001,Skill_A,0,0.0,9,Matutino
1,01/01/2024,09:30,0,0,0,0,0,0,0,1800,...,0,0,0,0,Agente_0001,Skill_A,0,0.0,9,Matutino
2,01/01/2024,10:00,0,0,0,0,0,0,0,1800,...,0,0,0,0,Agente_0001,Skill_A,0,0.0,10,Matutino
3,01/01/2024,10:30,0,0,0,0,0,0,0,1800,...,0,0,0,0,Agente_0001,Skill_A,0,0.0,10,Matutino
4,01/01/2024,11:00,0,0,0,0,0,0,0,451,...,0,0,133,0,Agente_0001,Skill_A,0,0.0,11,Matutino
